In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm.notebook import tqdm
import pandas as pd

import dask.dataframe as dd

In [ ]:
# Load video metadata

chunks_0 = []
chunks_1 = []
chunks_2 = []
chunks_3 = []
chunks_4 = []
chunks_5 = []
chunks_6 = []
chunks_7 = []
chunks_8 = []
chunks_9 = []
chunks_10 = []
chunks_11 = []
chunks_12 = []
chunks_13 = []


# Define the chunk size
chunk_size = 5000  # adjust based on memory availability

counter = 0

# Read the JSONL file in chunks
for chunk in tqdm(pd.read_json('YouNiverse\yt_metadata_en.jsonl.gz', lines=True, chunksize=chunk_size)):
    i = counter//1000
    globals()[f'chunks_{i}'].append(chunk)
    counter += 1


for i in tqdm(range(14)):
    name = f'df_video_meta_{i}'
    globals()[name] = pd.concat(globals()[f'chunks_{i}'], ignore_index=True)

# Concatenate all chunks into a single DataFrame after the loop
#df_video_meta = pd.concat(chunks, ignore_index=True)

0it [00:00, ?it/s]

In [ ]:
# Stopped running due to memory error
df_video_meta_dask = dd.read_json('YouNiverse/yt_metadata_en.jsonl.gz', lines=True)

MemoryError: An error occurred while calling the read_json method registered to the pandas backend.
Original Message: An error occurred while calling the read_json method registered to the pandas backend.
Original Message: 

In [3]:
# Define the chunk size (adjust based on memory availability)
chunk_size = 10000000

# Output Parquet file path (base name for individual files)
output_file_base = 'processed_data_chunk_'

# Initialize a counter for chunks
counter = 0

# Read the JSONL file in chunks
for chunk in tqdm(pd.read_json('YouNiverse/yt_metadata_en.jsonl.gz', lines=True, chunksize=chunk_size)):
    # Process each chunk as needed
    # Here you can apply any processing to the chunk

    # Save the chunk to a Parquet file
    chunk.to_parquet(f"{output_file_base}{counter}.parquet", index=False)

    # Increment the counter
    counter += 1
    print(f"Processed and saved chunk {counter}")

0it [00:00, ?it/s]

: 

In [ ]:
# First convert to parquet
import pyarrow.json as pj
import pyarrow.parquet as pq

# Read JSONL and convert to Parquet
input_file = "YouNiverse/yt_metadata_en.jsonl.gz"
output_file = "YouNiverse/yt_metadata.parquet"

table = pj.read_json(input_file)
pq.write_table(table, output_file)

# Now read the Parquet file with dask
import dask.dataframe as dd
df = dd.read_parquet(output_file)

# Ran for 37 min without success

In [1]:
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("LargeJSONProcessing").getOrCreate()

# Read the JSON file
df = spark.read.json("YouNiverse/yt_metadata_en.jsonl.gz")

# Convert to pandas DataFrame if necessary (or continue processing with Spark)
#pdf = filtered_df.toPandas()

RuntimeError: Java gateway process exited before sending its port number

In [2]:
import gzip
import json
from itertools import islice

# Define function to process a certain number of lines of the dataset
def process_jsonl_chunks(file_path, start_line=0, end_line=None, chunk_size=5000, output_parquet_file=None):
    """
    Reads a compressed JSON Lines file from a specific line range in chunks,
    processes each chunk, and optionally saves it to a Parquet file.

    Args:
        file_path (str): Path to the .jsonl.gz file.
        start_line (int): Line number to start reading from.
        end_line (int or None): Line number to stop reading at. If None, reads to the end.
        chunk_size (int): Number of lines per chunk.
        output_parquet_file (str): Path to save Parquet file. If None, does not save.

    Returns:
        None
    """
    
    with gzip.open(file_path, 'rt') as file:  # 'rt' for text mode with gzip
        # Skip to the start line
        print("Going to specified start line..")
        for _ in tqdm(range(start_line)):
            next(file)
        
        # Initialize a line counter
        current_line = start_line

        # Define dataframe to be returned
        df = []

        print("Getting chunks..")
        # Process the file in chunks until reaching end_line
        for i, chunk in enumerate(iter(lambda: list(islice(file, chunk_size)), [])):
            # Check if we're within the specified line range
            if end_line is not None and current_line >= end_line:
                break  # Stop if we've reached or exceeded the end line

            # Adjust the chunk size if it exceeds end_line
            if end_line is not None:
                chunk = chunk[: max(0, end_line - current_line)]
            
            # Convert the chunk to a DataFrame
            data = [json.loads(line) for line in chunk]
            df.append(pd.DataFrame(data))

            # Update current line
            current_line += len(chunk)
        
        print("Concatenating..")
        return pd.concat(df, ignore_index=True)

In [9]:
# Function to count lines of a JSON Lines file
def count_lines_in_chunks(file_path, chunk_size=100000):
    count = 0
    with gzip.open(file_path, 'rt') as file:
        while True:
            lines = file.readlines(chunk_size)  # Read a chunk of lines
            if not lines:
                break
            count += len(lines)
    return count

In [3]:
filepath = 'YouNiverse/yt_metadata_en.jsonl.gz'
#num_lines = count_lines_in_chunks(file_path=filepath)
#print(num_lines)
# There are as many lines as there are videos in the dataset

In [7]:
# End line is not included

# Loading data for first 20'000'000 videos
df_video_meta_1 = process_jsonl_chunks(file_path=filepath, start_line=0, end_line=20000000)

Going to specified start line..


0it [00:00, ?it/s]

Getting chunks..
Concatenating..


C:\Users\MaxLo\AppData\Local\Temp\ipykernel_26848\1242293564.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(df, ignore_index=True)


In [8]:
df_video_meta_1.tail(5)
df_video_meta_1.to_parquet(path='YouNiverse/df_video_meta_1.gz', compression='gzip')

In [7]:
df_video_meta_1_parquet = pd.read_parquet(path='YouNiverse/df_video_meta_1.gz')

In [9]:
df_video_meta_1_parquet.tail()

,categories,channel_id,crawl_date,description,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count
19999995,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:54.691949,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,8.0,jnfFTj85L8E,310,308.0,"02,Escape,From,The,Tavern,James,Horner",02 - Escape From The Tavern - James Horner - W...,2011-07-21 00:00:00,123004.0
19999996,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:55.296351,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,36.0,pRskpDvjrIM,590,1346.0,"01,Elora,Danan,James,Horner",01 - Elora Danan - James Horner - Willow,2011-07-21 00:00:00,308625.0
19999997,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:55.940956,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,2.0,SqggGvm_9a8,260,31.0,"14,See,You,James,Horner",14 - I See You - Leona Lewis - James Horner - ...,2011-06-23 00:00:00,3432.0
19999998,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:56.573328,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,0.0,Hjc5KhASGhI,680,21.0,"13,War,James,Horner",13 - War - James Horner - Avatar,2011-06-23 00:00:00,3590.0
19999999,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:57.067663,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,4.0,9j4OXdEJBpc,310,36.0,"12,Gathering,All,The,Na'vi,Clans,For,Battle,Ja...",12 - Gathering All The Na'vi Clans For Battle ...,2011-06-23 00:00:00,7562.0


In [10]:
df_video_meta_1.to_pickle(path='YouNiverse/df_video_meta_1_pickle.gz', compression='gzip')

In [ ]:
# Pickling and read pickle are slower than parqueting and read parquet -> preferably use parquet
df_video_meta_1_pickle = pd.read_pickle('YouNiverse/df_video_meta_1_pickle.gz')

In [6]:
df_video_meta_1_pickle.tail()

,categories,channel_id,crawl_date,description,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count
19999995,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:54.691949,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,8.0,jnfFTj85L8E,310,308.0,"02,Escape,From,The,Tavern,James,Horner",02 - Escape From The Tavern - James Horner - W...,2011-07-21 00:00:00,123004.0
19999996,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:55.296351,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,36.0,pRskpDvjrIM,590,1346.0,"01,Elora,Danan,James,Horner",01 - Elora Danan - James Horner - Willow,2011-07-21 00:00:00,308625.0
19999997,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:55.940956,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,2.0,SqggGvm_9a8,260,31.0,"14,See,You,James,Horner",14 - I See You - Leona Lewis - James Horner - ...,2011-06-23 00:00:00,3432.0
19999998,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:56.573328,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,0.0,Hjc5KhASGhI,680,21.0,"13,War,James,Horner",13 - War - James Horner - Avatar,2011-06-23 00:00:00,3590.0
19999999,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:57.067663,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,4.0,9j4OXdEJBpc,310,36.0,"12,Gathering,All,The,Na'vi,Clans,For,Battle,Ja...",12 - Gathering All The Na'vi Clans For Battle ...,2011-06-23 00:00:00,7562.0


In [5]:
# Loading videos from 20'000'000 to 39'999'999
df_video_meta_2 = process_jsonl_chunks(file_path=filepath, start_line=20000000, end_line=40000000)

Going to specified start line..


  0%|          | 0/20000000 [00:00<?, ?it/s]

Getting chunks..
Concatenating..


C:\Users\MaxLo\AppData\Local\Temp\ipykernel_7948\1242293564.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(df, ignore_index=True)


In [7]:
df_video_meta_2.head()

,categories,channel_id,crawl_date,description,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count
0,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:57.663245,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,0.0,7PoK8wW8KZo,170,12.0,"11,Shutting,Down,Grace's,Lab,James,Horner",11 - Shutting Down Grace's Lab - James Horner ...,2011-06-23 00:00:00,2152.0
1,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:58.193387,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,2.0,9yGTSMiexng,410,67.0,"10,The,Destruction,Of,Hometree,James,Horner",10 - The Destruction Of Hometree - James Horne...,2011-06-23 00:00:00,9772.0
2,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:58.698710,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,1.0,tJ6tOY5S8dA,300,20.0,"09,Quaritch,James,Horner",09 - Quaritch - James Horner - Avatar,2011-06-23 00:00:00,7495.0
3,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:59.253763,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,0.0,WPBk87tSOc4,200,7.0,"08,Scorched,Earth,James,Horner",08 - Scorched Earth - James Horner - Avatar,2011-06-23 00:00:00,1776.0
4,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:59.770408,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,0.0,UkbgdSZemWU,290,20.0,"07,Jake's,First,Flight,James,Horner",07 - Jake's First Flight - James Horner - Avatar,2011-06-23 00:00:00,3193.0


In [8]:
df_video_meta_2.to_parquet(path='YouNiverse/df_video_meta_2.gz', compression='gzip')

In [9]:
df_video_meta_2_parquet = pd.read_parquet(path='YouNiverse/df_video_meta_2.gz')

In [11]:
df_video_meta_2_parquet.tail()

,categories,channel_id,crawl_date,description,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count
19999995,Entertainment,UCMz7JFeTmiO2oJgtmYhpUmQ,2019-11-10 20:43:47.478141,"The night of the Star Wars Premiere, Jesse tri...",2497.0,TBPyu7liNQ0,1575,32750.0,"psycho series,psycho dad,psycho,mcjuggernugget...",I AM YOUR FATHER!,2015-12-17 00:00:00,1419226.0
19999996,Entertainment,UCMz7JFeTmiO2oJgtmYhpUmQ,2019-11-10 20:43:48.035144,"Hey there, Juggies and non-Juggies! Expect vid...",8343.0,uVMiG6CllbQ,267,33793.0,"psycho series,psycho dad,psycho,mcjuggernugget...","Hello, I am McJuggerNuggets! (2016)",2015-12-17 00:00:00,5110665.0
19999997,Entertainment,UCMz7JFeTmiO2oJgtmYhpUmQ,2019-11-10 20:43:48.556228,Jesse and Corn get roped into bringing his Mom...,3145.0,7n9EQuv_8uI,1701,49344.0,"psycho series,psycho dad,psycho,mcjuggernugget...",DRUNK DANCE NIGHT!,2015-12-16 00:00:00,1368194.0
19999998,Entertainment,UCMz7JFeTmiO2oJgtmYhpUmQ,2019-11-10 20:43:49.214078,"In this twenty-third installment of ""McJuggerN...",1440.0,l1Eh0TntG24,204,8336.0,"psycho series,psycho dad,psycho,mcjuggernugget...",#MMM23 -- Demons,2015-12-16 00:00:00,237093.0
19999999,Entertainment,UCMz7JFeTmiO2oJgtmYhpUmQ,2019-11-10 20:43:49.812247,Uncle Larry confronts Aunt Melissa about her r...,2437.0,_Slmlm21pRk,1656,33978.0,"psycho series,psycho dad,psycho,mcjuggernugget...",LIVING IN THE PAST!,2015-12-15 00:00:00,1536545.0


In [4]:
# Loading videos from 40'000'000 to 59'999'999
df_video_meta_3 = process_jsonl_chunks(file_path=filepath, start_line=40000000, end_line=60000000)

Going to specified start line..


  0%|          | 0/40000000 [00:00<?, ?it/s]

Getting chunks..
Concatenating..


C:\Users\MaxLo\AppData\Local\Temp\ipykernel_12444\1242293564.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(df, ignore_index=True)


In [5]:
df_video_meta_3.to_parquet(path='YouNiverse/df_video_meta_3.gz', compression='gzip')

In [10]:
df_video_meta_3_parquet = pd.read_parquet(path='YouNiverse/df_video_meta_3.gz')

ArrowMemoryError: malloc of size 4294967296 failed

In [9]:
df_video_meta_3_parquet.tail()

NameError: name 'df_video_meta_3_parquet' is not defined

In [4]:
# Loading videos from 60'000'000 to 72924794
df_video_meta_4 = process_jsonl_chunks(file_path=filepath, start_line=60000000, end_line=72924795)

Going to specified start line..


  0%|          | 0/60000000 [00:00<?, ?it/s]

Getting chunks..
Concatenating..


C:\Users\MaxLo\AppData\Local\Temp\ipykernel_8064\1242293564.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(df, ignore_index=True)


In [5]:
df_video_meta_4.to_parquet(path='YouNiverse/df_video_meta_4.gz', compression='gzip')

In [6]:
df_video_meta_4_parquet = pd.read_parquet(path='YouNiverse/df_video_meta_4.gz')

In [8]:
df_video_meta_4_parquet.tail()

,categories,channel_id,crawl_date,description,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count
12924789,News & Politics,UCrwE8kVqtIUVUzKui2WVpuQ,2019-11-01 23:46:02.294620,Shri Manoj Kumar Tiwari's speech during Motion...,3.0,YQLoxwLpjSU,270,67.0,"BJP,Bharatiya Janata Party,BJP videos,Yuva TV,...",Shri Manoj Kumar Tiwari's speech during Motion...,2017-02-06 00:00:00,4409.0
12924790,News & Politics,UCrwE8kVqtIUVUzKui2WVpuQ,2019-11-01 23:46:06.401481,Shri La Ganesan's speech during Motion of Than...,0.0,mINQHg1QBcg,878,21.0,"BJP,Bharatiya Janata Party,BJP videos,Yuva TV,...",Shri La Ganesan's speech during Motion of Than...,2017-02-06 00:00:00,1172.0
12924791,News & Politics,UCrwE8kVqtIUVUzKui2WVpuQ,2019-11-01 23:46:09.530822,Shri Mukhtar Abbas Naqvi's speech during Motio...,2.0,x20aNOWh1yI,1003,35.0,"BJP,Bharatiya Janata Party,BJP videos,Yuva TV,...",Shri Mukhtar Abbas Naqvi's speech during Motio...,2017-02-06 00:00:00,1898.0
12924792,News & Politics,UCrwE8kVqtIUVUzKui2WVpuQ,2019-11-01 23:46:00.080054,BJP submitted complaint to EC against Chief Se...,0.0,-Nn6FL2gqEw,755,27.0,"BJP,Bharatiya Janata Party,BJP videos,Yuva TV,...",BJP submitted complaint to EC against Chief Se...,2017-02-06 00:00:00,726.0
12924793,News & Politics,UCrwE8kVqtIUVUzKui2WVpuQ,2019-11-01 23:46:02.381286,Shri Amit Shah speech at public meeting in Noi...,31.0,7gxEjSoRVgA,1771,327.0,"BJP,Bharatiya Janata Party,BJP videos,Yuva TV,...",Shri Amit Shah speech at public meeting in Noi...,2017-02-06 00:00:00,37572.0


In [ ]:
# Load all parquet files into dask dataframes
dd_video_meta_1_parquet = dd.read_parquet(path='YouNiverse/df_video_meta_1.gz')
dd_video_meta_2_parquet = dd.read_parquet(path='YouNiverse/df_video_meta_2.gz')
dd_video_meta_3_parquet = dd.read_parquet(path='YouNiverse/df_video_meta_3.gz')
dd_video_meta_4_parquet = dd.read_parquet(path='YouNiverse/df_video_meta_4.gz')

In [4]:
df_test = pd.read_parquet(path='YouNiverse/df_video_meta_1.gz')

In [6]:
df_test.tail()

,categories,channel_id,crawl_date,description,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count
19999995,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:54.691949,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,8.0,jnfFTj85L8E,310,308.0,"02,Escape,From,The,Tavern,James,Horner",02 - Escape From The Tavern - James Horner - W...,2011-07-21 00:00:00,123004.0
19999996,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:55.296351,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,36.0,pRskpDvjrIM,590,1346.0,"01,Elora,Danan,James,Horner",01 - Elora Danan - James Horner - Willow,2011-07-21 00:00:00,308625.0
19999997,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:55.940956,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,2.0,SqggGvm_9a8,260,31.0,"14,See,You,James,Horner",14 - I See You - Leona Lewis - James Horner - ...,2011-06-23 00:00:00,3432.0
19999998,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:56.573328,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,0.0,Hjc5KhASGhI,680,21.0,"13,War,James,Horner",13 - War - James Horner - Avatar,2011-06-23 00:00:00,3590.0
19999999,Film & Animation,UCbzt7jamo0xA0WKPX3Ut-rw,2019-11-15 03:58:57.067663,FACEBOOK PAGE\nhttps://www.facebook.com/JamesH...,4.0,9j4OXdEJBpc,310,36.0,"12,Gathering,All,The,Na'vi,Clans,For,Battle,Ja...",12 - Gathering All The Na'vi Clans For Battle ...,2011-06-23 00:00:00,7562.0


In [ ]:
# Concatenate all dask dataframes into one
dfs = [dd_video_meta_1_parquet, dd_video_meta_2_parquet, dd_video_meta_3_parquet, dd_video_meta_4_parquet]
dd_parquet = dd.concat(dfs)

In [ ]:
# Write all dask dataframes into a Parquet file containing all the video metadata
dd.to_parquet(dd_parquet, path='YouNiverse/df_video_meta_parquet.gz', compression='gzip')

In [ ]:
df_video_meta = pd.read_parquet(path='YouNiverse/df_video_meta_parquet.gz')
# Crashed multiple times while running

MemoryError: Unable to allocate 1.00 MiB for an array with shape (131072,) and data type object

In [2]:
dd_video_meta = dd.read_parquet(path='YouNiverse/df_video_meta_parquet.gz')

In [3]:
dd_video_meta.loc[dd_video_meta.categories=='Music'].head()

,categories,channel_id,crawl_date,description,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count
2658,Music,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:42:09.015401,Link to the FULL music video: http://www.youtu...,28.0,WVapFh7cx3M,142,184.0,"En Av oss,one of us,project,Gutterommet,music,...",My Biggest Project EVER! Music Video - En Av O...,2016-06-06 00:00:00,4360.0
3053,Music,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:42:16.320706,Need Free Diamonds? [ Clash Of Clans - Boom B...,126.0,IyfriHeXo3I,35,1037.0,"boom beach,boom beach general,bootramp,riflema...",Boom Beach General Song,2015-06-04 00:00:00,43190.0
3220,Music,UCzWfqsjlvzAOUozsGFR3kbg,2019-10-31 15:41:52.783564,Thank you Nordic Screens Norway for giving me ...,112.0,O0N0fH41wGw,140,1193.0,"official,music,boom beach,boom beach song,boom...",BRAVE | Boom Beach | Bootramp feat Kosteofficial,2015-01-05 00:00:00,41416.0
3591,Music,UCzWdpFOflXTOk5Gsi2aJ67g,2019-11-13 21:46:26.995798,Subscribe for more horror music: https://www.y...,0.0,H20js4rsYeI,112,102.0,"horror music,best horror music,scary music,cre...",Horror Trailer Music - The Terror | Atmospheri...,2019-10-02 00:00:00,2643.0
3592,Music,UCzWdpFOflXTOk5Gsi2aJ67g,2019-11-13 21:46:27.615035,Subscribe for more horror music: https://www.y...,1.0,bM06MQ--Daw,157,63.0,"horror music,best horror music,scary music,cre...",Synapse Trailer Music - Padded Room | Epic Hyb...,2019-10-01 00:00:00,1304.0


In [3]:
dd_video_meta.loc[dd_video_meta.categories=='Entertainment'].head()

,categories,channel_id,crawl_date,description,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count
265,Entertainment,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:38.143766,Subscribe for more: https://goo.gl/JI78Ak New ...,0.0,6LvO-iX4FLw,193,10.0,"Hulk (Comic Book Character),Batman (Comic Book...",The Hulk wakes up by a Minions Batman DC Heroes,2015-11-01 00:00:00,865.0
270,Entertainment,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:29.608486,Subscribe for more: https://goo.gl/JI78Ak New ...,5.0,ztT0SeQ28Eg,422,13.0,"lego kids,lego pirates,lego animals,lego fores...",Lego Forest Movie for babies with animals boa...,2015-09-12 00:00:00,13846.0
271,Entertainment,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:30.199041,Subscribe for more: https://goo.gl/JI78Ak New ...,21.0,O2TGCiUb01Y,300,71.0,"lego circus,lego toys,lego juguetes,Lego train...",Lego Circus movie for kids with animals toys C...,2015-09-11 00:00:00,63887.0
272,Entertainment,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:30.800639,Subscribe for more: https://goo.gl/JI78Ak New ...,46.0,j1plTifI4i4,648,442.0,"lego kids,lego junior,lego fireman,lego city p...",Lego City police Lego fireman Cartoons about L...,2015-09-11 00:00:00,212832.0
273,Entertainment,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:31.440881,Lego Policia del pantano 60069 Aqui tenemos la...,5.0,1Gvhx_KzSf0,676,39.0,"lego,bricomania,piezas lego,lego policia,lego ...",Lego Policia del pantano 60069 Swamp Police,2015-09-04 00:00:00,12625.0
